# **Free-Electron Bands in a Periodic Lattice**

**Authors:** Dou Du, Taylor James Baird and Giovanni Pizzi

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/free_electron.ipynb

  The main objective of this notebook is to demonstrate the electronic bandstructure within the free-electron model for a periodic crystalline lattice of a metal.
  
     
Throughout the notebook, we employ the empty lattice (free-electron) approximation for the electrons in a periodic 
solid system.  Using it, we compute and plot the electronic band structure for three 
types of Bravais lattice: simple cubic (SC), face-centered cubic (FCC) and body-centered cubic (BCC). We get the path in reciprocal space for the band structure  
from the <a href="https://seekpath.readthedocs.io/en/latest/index.html">seekpath</a>
package.

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

* Familiarize yourself with the free-electron model of a metallic solid.
* Examine the electronic band structure of the free-electron model for different crystalline structures.


## **Background theory**
   
[More on the background theory.](./theory/theory_free_electron.ipynb)

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
     <li> Can you describe the shape of the band structure in the 1st Brillouin zone?
     <details>
    <summary style="color: red">Solution</summary>
    In the free electron model, the dispersion relation between electronic energy and wavevector is given by $E=\frac{\hbar^2k^2}{2m}$. Accordingly, the shape of the bands is parabolic. 
    </details>   
    </li>
     <li> What properties of a material shall be best captured by the free-electron model?
     <details>
    <summary style="color: red">Solution</summary>
    As the free-electron model neglects the effect of the ionic potential on the electrons, material properties which are primarily dependent on the kinetic energy of the conduction electrons are those which shall be best described by the model.
    </details>   
    </li>
      <li> Consider the simple cubic lattice, and consider the lowest energy band from Γ to the X point, from Γ to the R point, and from Γ to the M point. The curvature of those bands seem to be the same. Is this to be expected? What about the Γ-L, Γ-X, and Γ-K in the FCC case? Or the Γ-H, Γ-N, and Γ-P in the BCC case?
     <details>
    <summary style="color: red">Solution</summary>
    For a free electron case, the bands are isotropic (i.e., they are the same, independent of the direction in k space), and the effective mass is just the free-electron mass: $m^*=m_0$. Therefore, we expect all curvatures (of the lowest energy band starting from Γ) to be the same.
    </details>   
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
import numpy as np
import seekpath
import re
from widget_bzvisualizer import BZVisualizer

In [ ]:
def prettify(label):
    """
    Prettifier for matplotlib, using LaTeX syntax
    :param label: a string to prettify
    """

    label = (
        label
            .replace('GAMMA', r'$\Gamma$')
            .replace('DELTA', r'$\Delta$')
            .replace('LAMBDA', r'$\Lambda$')
            .replace('SIGMA', r'$\Sigma$')
    )
    label = re.sub(r'_(.?)', r'$_{\1}$', label)

    return label

In [ ]:
def _get_band_energies(kpoints_list, b1, b2, b3, g_vectors_range):
    energy_data_curves = np.zeros(((2*g_vectors_range+1)**3, len(kpoints_list)), dtype=np.float_)

    cnt = 0
    for g_i in range(-g_vectors_range,g_vectors_range+1):
        for g_j in range(-g_vectors_range,g_vectors_range+1):
            for g_k in range(-g_vectors_range,g_vectors_range+1):
                g_vector = b1 * g_i + b2*g_j + b3 * g_k
                energy_data_curves[cnt] = np.sum(0.5*(kpoints_list + g_vector)**2, axis=1)# This is k^2 - NOTE: units to be double checked!
                cnt += 1


    # bands are ordered as follows: first band, second band, ...
    return energy_data_curves

def _compute_dos(kpts, G, ranges):
    eigs = []
    n = ranges
    
    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                eigs.append(np.sum(0.5*(kpts + g_vector)**2, axis=3))

    eigs = np.moveaxis(eigs, 0, -1)
    return eigs
   
def _compute_total_kpts(kpts, G, ranges):
    tot_kpts = []
    n = ranges

    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                tot_kpts.extend(kpts+g_vector)
    return np.array(tot_kpts)
    

In [ ]:
def get_bands(real_lattice_bohr, reference_distance = 0.05, g_vectors_range = 3):
    """Function to return the band structure of a free-electron model.
    
    :param real_lattice_bohr: 3x3 lattice vectors of the crystal, in bohr
    :param reference_distance: distance between consecutive k-points in the band structure.
    :param g_vectors_range: integer indicating how many G vectors should be included
        to compute the band structure.
    """
    
    # Simple way to get automatically the band path:
    # I go back to real space, just put a single atom at the origin,
    # then go back with seekpath.
    # NOTE! This might not give the most general path, as e.g. there are two
    # options for cubic FCC (cF1 and cF2 in seekpath).
    # But this should be general enough for this tool.

    structure = (real_lattice_bohr, [[0., 0., 0.]], [1])
    # Use a H atom at the origin
    seekpath_path = seekpath.get_explicit_k_path(structure, reference_distance=reference_distance)
    b1, b2, b3 = np.array(seekpath_path['reciprocal_primitive_lattice'])

    all_kpoints_x = np.array(seekpath_path['explicit_kpoints_linearcoord'])
    all_kpoints_list = np.array(seekpath_path['explicit_kpoints_abs'])

    segments_data = []
    for segment_indices in seekpath_path['explicit_segments']:
        start_label = seekpath_path['explicit_kpoints_labels'][segment_indices[0]]
        end_label = seekpath_path['explicit_kpoints_labels'][segment_indices[1]-1]

        kpoints_x = all_kpoints_x[slice(*segment_indices)]
        kpoints_list = all_kpoints_list[slice(*segment_indices)]

        energy_bands = _get_band_energies(kpoints_list, b1, b2, b3, g_vectors_range)

        segments_data.append({
            'start_label': start_label,
            'end_label': end_label,
            'kpoints_list': kpoints_list,
            'kpoints_x': kpoints_x,
            'energy_bands': energy_bands,
            'b1': b1,
            'b2': b2,
            'b3': b3,
        })

    return segments_data

In [ ]:
from ipywidgets import RadioButtons, HBox, VBox

alat_bohr = 7.72

lattices = np.zeros((3, 3, 3))

lattices[0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * alat_bohr / 2.0
lattices[1] = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) * alat_bohr / 2.0
lattices[2] = np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) * alat_bohr / 2.0

real_lattice_bohr = lattices[0]
bz = BZVisualizer(real_lattice_bohr, [[0.0, 0.0, 0.0]], [1], height='400px', show_pathpoints = True, disable_interact_overlay = True)

In [ ]:
def unicode_gamma(label):
    if label == "GAMMA":
        return 'Γ'
    return label

def free_electron_aiida_bands(segments_data):
    """
    Turns the free electron bandstructure segments data into
    the AiiDA format that can be used with the widget-bandsplot
    """
    path = []
    paths = []
    for seg in segments_data:
        from_label = unicode_gamma(seg["start_label"])
        to_label = unicode_gamma(seg["end_label"])
        path.append([from_label, to_label])
        paths.append({
            "length": len(seg['kpoints_x']),
            "from": from_label,
            "to": to_label,
            "two_band_types": False,
            "x": seg["kpoints_x"].tolist(),
            "values": seg["energy_bands"].tolist(),
        })

    return {
        "label": "",
        "path": path,
        "paths": paths,
        #"fermi_level": 0,
    }

style = {'description_width': 'initial'}
cell_type = RadioButtons(options=['Simple cubic', 'FCC', 'BCC'], value='Simple cubic', description="Cell type:")

def on_celltype_changed(c):
    global real_lattice_bohr
    real_lattice_bohr = lattices[cell_type.index]
    bz.cell = real_lattice_bohr.tolist()
    print("test")
    
    plot_bandstructure()

cell_type.observe(on_celltype_changed, names='value')

def plot_bandstructure():
    segments_data = get_bands(real_lattice_bohr)
    aiida_bands = free_electron_aiida_bands(segments_data)
    bands_widget.bands = [aiida_bands]


from widget_bandsplot import BandsPlotWidget

bands_widget = BandsPlotWidget(
    energy_range = [0.0, 5.0],
    format_settings = {"bandsYlabel": "Free electron energy (eV)"}
)
plot_bandstructure()

display(HBox([VBox([bz,cell_type]), bands_widget]))

## Legend

<p style="text-align: justify;font-size:15px">
    The 1st Brillouin zone of the selected cell is shown on the left. 
    The path along which the band structure is calculated is indicated with blue vectors and sampled
    k-points are shown with red dots.
    The figure on the right shows the calculated band structure. 
    We provide three kinds of cell structure: simple cubic, 
    face-centered cubic (FCC) and body-centered cubic (BCC). Use the radio 
    button to select the cell type.
</p>
    